### OCI Data Science - Useful Tips
<details>
<summary><font size="2">Check for Public Internet Access</font></summary>

```python
import requests
response = requests.get("https://oracle.com")
assert response.status_code==200, "Internet connection failed"
```
</details>
<details>
<summary><font size="2">Helpful Documentation </font></summary>
<ul><li><a href="https://docs.cloud.oracle.com/en-us/iaas/data-science/using/data-science.htm">Data Science Service Documentation</a></li>
<li><a href="https://docs.cloud.oracle.com/iaas/tools/ads-sdk/latest/index.html">ADS documentation</a></li>
</ul>
</details>
<details>
<summary><font size="2">Typical Cell Imports and Settings for ADS</font></summary>

```python
%load_ext autoreload
%autoreload 2
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

import logging
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.ERROR)

import ads
from ads.dataset.factory import DatasetFactory
from ads.automl.provider import OracleAutoMLProvider
from ads.automl.driver import AutoML
from ads.evaluations.evaluator import ADSEvaluator
from ads.common.data import ADSData
from ads.explanations.explainer import ADSExplainer
from ads.explanations.mlx_global_explainer import MLXGlobalExplainer
from ads.explanations.mlx_local_explainer import MLXLocalExplainer
from ads.catalog.model import ModelCatalog
from ads.common.model_artifact import ModelArtifact
```
</details>
<details>
<summary><font size="2">Useful Environment Variables</font></summary>

```python
import os
print(os.environ["NB_SESSION_COMPARTMENT_OCID"])
print(os.environ["PROJECT_OCID"])
print(os.environ["USER_OCID"])
print(os.environ["TENANCY_OCID"])
print(os.environ["NB_REGION"])
```
</details>

In [1]:
import pandas as pd
import numpy as np
import pickle

In [5]:
fn = '/home/datascience/WorkSpace/RedBull/data/laps_2021.pkl'
file = open(fn, 'rb')
laps = pickle.load(file)

In [7]:
Laps = pd.concat(laps)

In [15]:
Laps[Laps['Team']=='Red Bull Racing']

,Time,DriverNumber,LapTime,LapNumber,Stint,PitOutTime,PitInTime,Sector1Time,Sector2Time,Sector3Time,...,LapStartTime,Team,Driver,TrackStatus,IsAccurate,LapStartDate,EventName,country,session,EventDate
0,0 days 00:21:18.434000,33,NaT,1.0,1.0,NaT,NaT,NaT,NaT,NaT,...,NaT,Red Bull Racing,VER,1,False,NaT,Bahrain Grand Prix,Bahrain,Qualifying,2021-03-28
1,0 days 00:24:04.661000,33,NaT,2.0,1.0,0 days 00:21:18.434000,NaT,NaT,0 days 00:01:24.332000,0 days 00:00:45.317000,...,0 days 00:21:18.434000,Red Bull Racing,VER,1,False,2021-03-27 15:06:18.819,Bahrain Grand Prix,Bahrain,Qualifying,2021-03-28
2,0 days 00:25:35.160000,33,0 days 00:01:30.499000,3.0,1.0,NaT,NaT,0 days 00:00:28.807000,0 days 00:00:38.980000,0 days 00:00:22.712000,...,0 days 00:24:04.661000,Red Bull Racing,VER,1,True,2021-03-27 15:09:05.046,Bahrain Grand Prix,Bahrain,Qualifying,2021-03-28
3,0 days 00:27:25.520000,33,0 days 00:01:50.360000,4.0,2.0,NaT,0 days 00:27:23.844000,0 days 00:00:34.462000,0 days 00:00:43.825000,0 days 00:00:32.073000,...,0 days 00:25:35.160000,Red Bull Racing,VER,1,False,2021-03-27 15:10:35.545,Bahrain Grand Prix,Bahrain,Qualifying,2021-03-28
4,0 days 00:44:22.361000,33,NaT,5.0,2.0,0 days 00:42:21.384000,NaT,NaT,0 days 00:00:54.163000,0 days 00:00:28.730000,...,0 days 00:27:25.520000,Red Bull Racing,VER,2,False,2021-03-27 15:12:25.905,Bahrain Grand Prix,Bahrain,Qualifying,2021-03-28
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
859,0 days 02:18:56.779000,11,0 days 00:01:26.419000,51.0,3.0,NaT,NaT,0 days 00:00:17.435000,0 days 00:00:37.485000,0 days 00:00:31.499000,...,0 days 02:17:30.360000,Red Bull Racing,PER,1,True,2021-12-12 14:18:30.370,Abu Dhabi Grand Prix,UAE,Race,2021-12-12
860,0 days 02:20:23.308000,11,0 days 00:01:26.529000,52.0,3.0,NaT,NaT,0 days 00:00:17.498000,0 days 00:00:37.487000,0 days 00:00:31.544000,...,0 days 02:18:56.779000,Red Bull Racing,PER,1,True,2021-12-12 14:19:56.789,Abu Dhabi Grand Prix,UAE,Race,2021-12-12
861,0 days 02:22:10.915000,11,0 days 00:01:47.607000,53.0,3.0,NaT,0 days 02:22:09.875000,0 days 00:00:17.423000,0 days 00:00:48.122000,0 days 00:00:42.062000,...,0 days 02:20:23.308000,Red Bull Racing,PER,24,False,2021-12-12 14:21:23.318,Abu Dhabi Grand Prix,UAE,Race,2021-12-12
862,0 days 02:24:17.809000,11,0 days 00:02:06.894000,54.0,4.0,0 days 02:22:31.269000,NaT,0 days 00:00:38.909000,0 days 00:00:46.980000,0 days 00:00:41.005000,...,0 days 02:22:10.915000,Red Bull Racing,PER,4,False,2021-12-12 14:23:10.925,Abu Dhabi Grand Prix,UAE,Race,2021-12-12


array(['Alfa Romeo Racing', 'AlphaTauri', 'Alpine', 'Aston Martin',
       'Ferrari', 'Haas F1 Team', 'McLaren', 'Mercedes',
       'Red Bull Racing', 'Williams'], dtype=object)